In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

#### re-labeling

In [19]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [45]:
modelname='bash1130/bert-base-finetuned-ynat' #klue/bert-base 
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSequenceClassification.from_pretrained(modelname,num_labels=7).to(DEVICE)

In [46]:
df=pd.read_csv('./Relabel_data_naver_crawler.csv')
#df=df.drop(columns=['url','date','ID','Unnamed: 0'])

In [48]:
preds=[]
#re-label the text 
for i in tqdm(range(len(df))):
    text = df.loc[i,'text']
    tokens = tokenizer(text,return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        logits = model(**tokens).logits
        pred = torch.argmax(torch.nn.Softmax(dim=1)(logits), dim=1).cpu().numpy()
        preds.extend(pred)

100%|██████████| 1300/1300 [00:31<00:00, 41.65it/s]


In [49]:
df['target'] = preds

In [50]:
df.to_csv('naver.csv', index=False)